In [1]:
import requests
import json
import io
import numpy as np
import re

## Insert data

In [140]:
ts1 = '{"1": {"ts": {"times": [1, 2, 3], "values": [2, 4, 9]}}}'
ts2 = '{"2": {"ts": {"times": [1, 2, 3], "values": [1, 5, 10]}}}'

In [141]:
r = requests.post('http://127.0.0.1:5000/insert', json = ts1)
r = requests.post('http://127.0.0.1:5000/insert', json = ts2)

## Request data

In [142]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))

In [139]:
r.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"\n  "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n  <head>\n    <title>OSError: Multiple exceptions: [Errno 61] Connect call failed (\'::1\', 30000, 0, 0), [Errno 61] Connect call failed (\'127.0.0.1\', 30000) // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"\n        type="text/css">\n    <!-- We need to make sure this has a favicon so that the debugger does\n         not by accident trigger a request to /favicon.ico which might\n         change the application state. -->\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script type="text/javascript">\n      var TRACEBACK = 4400921400,\n          CONSOLE_MODE = false,\n          EVALEX = tru

In [7]:
assert r.text == '<pre>{\n    "1": {},\n    "2": {}\n}</pre>' or '<pre>{\n    "2": {},\n    "1": {}\n}</pre>'

In [925]:
r = requests.get('http://127.0.0.1:5000/select?fields=ts', auth=('user', 'pass'))
# r.json()

## Insert meta

In [21]:
meta = {'1': {'order': 1, 'blarg': 1}}
meta2 = {'2': {'order': 2, 'blarg': 2}}

#  db.upsert_meta('one', {'order': 1, 'blarg': 1})
# print(meta)
s_meta = io.StringIO()
json.dump(meta, fp=s_meta)
s_meta.getvalue()

s_meta2 = io.StringIO()
json.dump(meta2, fp=s_meta2)
s_meta2.getvalue()

'{"2": {"blarg": 2, "order": 2}}'

In [22]:
r = requests.post('http://127.0.0.1:5000/upsert_meta', json = s_meta.getvalue())
r = requests.post('http://127.0.0.1:5000/upsert_meta', json = s_meta2.getvalue())

In [10]:
r = requests.get('http://127.0.0.1:5000/select?fields=order', auth=('user', 'pass'))
r.text

'<pre>{\n    "2": {\n        "order": 2\n    },\n    "1": {\n        "order": 1\n    }\n}</pre>'

In [31]:
assert r.json() == {'1': {'order': 1}, '2': {'order': 2}}

## Sort by order

In [12]:
r = requests.get('http://127.0.0.1:5000/select?additional=sort_by,%2Border', auth=('user', 'pass'))
r.text

'<pre>{\n    "1": {},\n    "2": {}\n}</pre>'

In [399]:
r = requests.get('http://127.0.0.1:5000/select?additional=sort_by,%2Border,limit,1', auth=('user', 'pass'))
r.json()

{'10': {'blarg': 1, 'order': 1, 'pk': '10'}}

In [53]:
jsonfile = r.json()
jsonfile

{'10': {'ts': {'times': [1, 2, 3], 'values': [4, 5, 6]}}}

In [194]:
s = io.StringIO()
json.dump(jsonfile, fp=s)
s.getvalue()

'{"10": {"ts": {"values": [4, 5, 6], "times": [1, 2, 3]}}}'

In [260]:
other_value = '{"10": {"ts": {"times": [1, 2, 3], "values": [2, 4, 9]}}}'

In [267]:
r = requests.post('http://127.0.0.1:5000/insert', json = s.getvalue())

In [81]:
s.getvalue()

'{"10": {"ts": {"values": [4, 5, 6], "times": [1, 2, 3]}}}'

In [86]:
data = json.loads(s.getvalue())

In [643]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))
r.json()

{'10': {'pk': '10'}, '12': {'pk': '12'}}

## Filter by meta

In [689]:
r = requests.get('http://127.0.0.1:5000/select?md=blarg,<2', auth=('user', 'pass'))
# {'blarg': {'<=': 2}}
r.json()

{}

## Select augmented

In [54]:
augmented = {'md': {"blarg": {'>=': 1}},'proc':'stats','target':['damean','dastd'], 'additional':{"sort_by": "+order", "limit": 2}}
augemented_json = io.StringIO()
json.dump(augmented, fp=augemented_json)
augmented_data = augemented_json.getvalue()
augmented_data

'{"md": {"blarg": {">=": 1}}, "target": ["damean", "dastd"], "proc": "stats", "additional": {"limit": 2, "sort_by": "+order"}}'

In [56]:
r = requests.post('http://127.0.0.1:5000/augmented_select', json = augmented_data)
# print (r.text)
r.text

'{\n    "1": {\n        "damean": 5.0,\n        "dastd": 2.943920288775949\n    },\n    "2": {\n        "damean": 5.333333333333333,\n        "dastd": 3.6817870057290873\n    }\n}'

## Delete ts

In [86]:
requests.post('http://127.0.0.1:5000/delete', json = '2')

In [ ]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))

## Find similar

In [116]:
values = np.random.randint(1,100,100)
times = np.arange(1,101)

In [117]:
ts =  {'times': times.tolist(), 'values': values.tolist()}

#  db.upsert_meta('one', {'order': 1, 'blarg': 1})
# print(meta)
ts_a = io.StringIO()
json.dump( ts, fp=ts_a)
ts_a.getvalue()


'{"times": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], "values": [53, 66, 30, 98, 23, 44, 10, 53, 35, 61, 58, 33, 85, 51, 25, 84, 62, 33, 48, 87, 1, 60, 54, 69, 47, 53, 12, 91, 76, 61, 26, 85, 93, 22, 70, 2, 87, 20, 78, 50, 92, 16, 27, 96, 77, 34, 62, 40, 78, 35, 23, 6, 28, 84, 81, 86, 25, 33, 6, 37, 45, 12, 74, 78, 36, 59, 33, 80, 7, 59, 83, 3, 2, 14, 88, 49, 10, 4, 80, 97, 2, 91, 89, 35, 19, 28, 47, 34, 50, 31, 35, 77, 9, 42, 11, 82, 78, 44, 21, 78]}'

In [118]:
r = requests.post('http://127.0.0.1:5000/find_similar', json = ts_a.getvalue())

In [113]:
r.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"\n  "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n  <head>\n    <title>TimeoutError // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"\n        type="text/css">\n    <!-- We need to make sure this has a favicon so that the debugger does\n         not by accident trigger a request to /favicon.ico which might\n         change the application state. -->\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script type="text/javascript">\n      var TRACEBACK = 4400651680,\n          CONSOLE_MODE = false,\n          EVALEX = true,\n          EVALEX_TRUSTED = false,\n          SECRET = "WhxUfwcpdkSNvHnheTaq";\n    </script>\n  </head>\n  <body>\n    <di